In [111]:
desired_system_capacity = 750
option_table_path = 'example-diesel-generator-by-capacity.csv'
capacity_column = 'capacity_in_kw'

In [112]:
from pandas import read_csv
option_table = read_csv(option_table_path)
option_table

,Capacity in kW,Raw Cost,Installation Cost as Percent of Raw Cost,Maintenance Cost per Year as Percent of Raw Cost,Lifetime in Years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [113]:
from invisibleroads_macros.table import normalize_column_name
option_table.columns = [normalize_column_name(x, '_') for x in option_table.columns]
option_table.columns

Index([u'capacity_in_kw', u'raw_cost',
       u'installation_cost_as_percent_of_raw_cost',
       u'maintenance_cost_per_year_as_percent_of_raw_cost',
       u'lifetime_in_years'],
      dtype='object')

In [114]:
desired_system_capacity

750

In [115]:
x = (option_table['capacity_in_kw'] - desired_system_capacity).abs()
x

0    250
1    250
Name: capacity_in_kw, dtype: int64

In [116]:
option_table.ix[x.argsort()]

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [117]:
# Get two closest
aaa = option_table.ix[x.argsort()][:2]
aaa

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [118]:
a, b = option_table.ix[x.argsort()][:2]

ValueError: too many values to unpack

In [42]:
x.argsort()

0    0
1    1
Name: capacity_in_kw, dtype: int64

In [43]:
y = x.argsort()

In [44]:
y.values

array([0, 1])

In [61]:
from pandas import DataFrame
t = DataFrame([[1000]], columns=['a'])

In [62]:
x, y = t.a.argsort()[:2]

ValueError: need more than 1 value to unpack

In [59]:
x

1

In [60]:
y

2

In [55]:
from pandas import DataFrame
t = DataFrame([[1000], [500], [750]], columns=['a'])

In [56]:
t

,a
0,1000
1,500
2,750


In [57]:
t.a.argsort().values

array([1, 2, 0])

In [32]:
aaa.capacity_in_kw.values

array([ 500, 1000])

In [33]:
aaa.capacity_in_kw.values[1] - aaa.capacity_in_kw.values[0]

500

In [34]:
# Compute fraction of difference
desired_system_capacity - aaa.capacity_in_kw.values[0]

250

In [36]:
(desired_system_capacity - aaa.capacity_in_kw.values[0]) / float(aaa.capacity_in_kw.values[1] - aaa.capacity_in_kw.values[0])

0.5

In [37]:
aaa.ix[aaa.index[1]] - aaa.ix[aaa.index[0]]

capacity_in_kw                                        500
raw_cost                                            47500
installation_cost_as_percent_of_raw_cost                0
maintenance_cost_per_year_as_percent_of_raw_cost        0
lifetime_in_years                                       0
dtype: int64

In [38]:
(aaa.ix[aaa.index[1]] - aaa.ix[aaa.index[0]]) * 0.5

capacity_in_kw                                        250.0
raw_cost                                            23750.0
installation_cost_as_percent_of_raw_cost                0.0
maintenance_cost_per_year_as_percent_of_raw_cost        0.0
lifetime_in_years                                       0.0
dtype: float64

In [39]:
aaa.ix[aaa.index[0]] + (aaa.ix[aaa.index[1]] - aaa.ix[aaa.index[0]]) * 0.5

capacity_in_kw                                        750.0
raw_cost                                            76250.0
installation_cost_as_percent_of_raw_cost               25.0
maintenance_cost_per_year_as_percent_of_raw_cost        1.0
lifetime_in_years                                       5.0
dtype: float64

In [ ]:
# Spline it

In [16]:
t = option_table.ix[x.argsort()]
selected_system = t.ix[t.index[0]]
selected_system

capacity_in_kw                                        500
raw_cost                                            52500
installation_cost_as_percent_of_raw_cost               25
maintenance_cost_per_year_as_percent_of_raw_cost        1
lifetime_in_years                                       5
Name: 0, dtype: int64

In [40]:
# We should pre sort the tables to normalize them
option_table.sort_values([
    'capacity_in_kw',
], ascending=True)

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [52]:
option_table.sort_values('capacity_in_kw', ascending=False`)

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years
1,1000,100000,25,1,5
0,500,52500,25,1,5


In [ ]:
# We should assert non-duplicates of capacity_in_kw

In [43]:
len(set(option_table['capacity_in_kw']))

2

In [49]:
len(option_table)

2

In [50]:
from infrastructure_planning.exceptions import InvalidData

In [ ]:
if len(option_table) != len(set(option_table['capacity_in_kw'])):
    raise InvalidData

In [48]:
assert 1 == 2, 'hey'

AssertionError: hey

In [17]:
selected_system

capacity_in_kw                                        500
raw_cost                                            52500
installation_cost_as_percent_of_raw_cost               25
maintenance_cost_per_year_as_percent_of_raw_cost        1
lifetime_in_years                                       5
Name: 0, dtype: int64

In [18]:
# Be conservative (over-estimate price)
raw_cost_per_unit_capacity = selected_system['raw_cost'] / float(selected_system[capacity_column])
raw_cost_per_unit_capacity

105.0

In [19]:
raw_cost = desired_system_capacity * raw_cost_per_unit_capacity
raw_cost

78750.0

In [21]:
installation_cost = raw_cost * selected_system['installation_cost_as_percent_of_raw_cost'] / float(100)
installation_cost

19687.5

In [22]:
maintenance_cost_per_year = raw_cost * selected_system[
    'maintenance_cost_per_year_as_percent_of_raw_cost'] / float(100)
maintenance_cost_per_year

787.5

In [25]:
from invisibleroads_macros.math import divide_safely
from infrastructure_planning.exceptions import ExpectedPositive
replacement_cost_per_year = divide_safely(
    installation_cost, selected_system['lifetime_in_years'],
    ExpectedPositive('lifetime_in_years'))
replacement_cost_per_year

3937.5

In [26]:
from math import ceil

In [28]:
int(ceil(desired_system_capacity))

750

In [67]:
difference = option_table[capacity_column] - desired_system_capacity
sorted_indices = difference.abs().argsort()
index0 = sorted_indices[0]
index1 = sorted_indices[1] if len(sorted_indices) > 1 else index1

In [68]:
index0, index1

(0, 1)

In [71]:
# Compute fraction of difference in capacity
capacity0 = option_table[capacity_column].ix[index0]
capacity0

500

In [73]:
capacity1 = option_table[capacity_column].ix[index1]
capacity1

1000

In [75]:
fraction = divide_safely(capacity0, capacity1, ExpectedPositive(capacity_column))
fraction

0.5

In [80]:
t = option_table.copy()
t

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [96]:
t[capacity_column]

0     500
1    1000
Name: capacity_in_kw, dtype: int64

In [99]:
t['raw_cost_per_unit_capacity'] = t['raw_cost'].divide(t[capacity_column])

In [100]:
t

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years,raw_cost_per_unit_capacity
0,500,52500,25,1,5,105.0
1,1000,100000,25,1,5,100.0


In [102]:
from pandas import DataFrame
z = DataFrame([[1, 2]], columns=['a', 'b'])
z

,a,b
0,1,2


In [104]:
z.a / z.b

0    0.5
dtype: float64

In [84]:
float('inf') in (t['raw_cost'] / 0).values

True

In [85]:
from pandas import Series

In [94]:
len(0)

TypeError: object of type 'int' has no len()

In [86]:
import numpy as np

In [88]:
x = np.array([1, 0, 2])

In [89]:
0 in x

True

In [90]:
0 in t['raw_cost']

True

In [93]:
t.divide?

In [91]:
0 in 0

TypeError: argument of type 'int' is not iterable

In [78]:
option_table.ix[index0] + (option_table.ix[index1] - option_table.ix[index0]) * fraction

capacity_in_kw                                        750.0
raw_cost                                            76250.0
installation_cost_as_percent_of_raw_cost               25.0
maintenance_cost_per_year_as_percent_of_raw_cost        1.0
lifetime_in_years                                       5.0
dtype: float64

In [ ]:
# x + (y - x) * a
# x + ay - ax
# (1 - a)x + ay

In [ ]:
# Get two closest
aaa = option_table.ix[][:2]
aaa.capacity_in_kw.values[1] - aaa.capacity_in_kw.values[0]
# Compute fraction of difference
desired_system_capacity - aaa.capacity_in_kw.values[0]
(desired_system_capacity - aaa.capacity_in_kw.values[0]) / float(aaa.capacity_in_kw.values[1] - aaa.capacity_in_kw.values[0])

aaa.ix[aaa.index[1]] - aaa.ix[aaa.index[0]]
(aaa.ix[aaa.index[1]] - aaa.ix[aaa.index[0]]) * 0.5
aaa.ix[aaa.index[0]] + (aaa.ix[aaa.index[1]] - aaa.ix[aaa.index[0]]) * 0.5

In [ ]:
# def interpolate_system_cost_parameters
# def interpolate_system_cost_pack

In [119]:
def interpolate_system_cost_parameters(desired_system_capacity, option_table, capacity_column):
    """
    Interpolate parameters for computing system cost.
    Assume that the option_table has unique capacity values and at least one row.
    """
    # Get the indices of the two systems that are nearest in capacity
    difference = option_table[capacity_column] - desired_system_capacity
    sorted_indices = difference.abs().argsort()
    index0 = sorted_indices[0]
    index1 = sorted_indices[1] if len(sorted_indices) > 1 else index0
    # Add raw_cost_per_unit_capacity
    t = option_table.copy()
    t['raw_cost_per_unit_capacity'] = t['raw_cost'] / t[capacity_column]
    # Compute fraction of difference in capacity
    capacity0 = option_table[capacity_column].ix[index0]
    capacity1 = option_table[capacity_column].ix[index1]
    fraction = divide_safely(capacity0, capacity1, ExpectedPositive(capacity_column))
    # Interpolate
    x = t.ix[index0] + (t.ix[index1] - t.ix[index0]) * fraction    
    return {
        'raw_cost_per_unit_capacity': x['raw_cost_per_unit_capacity'],
        'installation_cost_as_percent_of_raw_cost': x['installation_cost_as_percent_of_raw_cost'],
        'maintenance_cost_per_year_as_percent_of_raw_cost': x['maintenance_cost_per_year_as_percent_of_raw_cost'],
        'lifetime_in_years': x['lifetime_in_years'],
    }

interpolate_system_cost_parameters(desired_system_capacity, option_table, capacity_column)

{'installation_cost_as_percent_of_raw_cost': 25.0,
 'lifetime_in_years': 5.0,
 'maintenance_cost_per_year_as_percent_of_raw_cost': 1.0,
 'raw_cost_per_unit_capacity': 102.5}

In [106]:
option_table

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [107]:
option_table['x'] = option_table.raw_cost / option_table.capacity_in_kw

In [108]:
option_table

,capacity_in_kw,raw_cost,installation_cost_as_percent_of_raw_cost,maintenance_cost_per_year_as_percent_of_raw_cost,lifetime_in_years,x
0,500,52500,25,1,5,105.0
1,1000,100000,25,1,5,100.0


In [ ]:
from pandas import read_csv


desired_system_capacity = 750
option_table_path = 'example-diesel-generator-by-capacity.csv'
capacity_column = 'capacity_in_kw'



option_table = read_csv(option_table_path)


def prepare_system_cost(desired_system_capacity, option_table, capacity_column):
    
    t = option_table.ix[]
    
    
# Get two closest
aaa = option_table.ix[][:2]
aaa
aaa.capacity_in_kw.values[1] - aaa.capacity_in_kw.values[0]
# Compute fraction of difference
desired_system_capacity - aaa.capacity_in_kw.values[0]
(desired_system_capacity - aaa.capacity_in_kw.values[0]) / float(aaa.capacity_in_kw.values[1] - aaa.capacity_in_kw.values[0])

aaa.ix[aaa.index[1]] - aaa.ix[aaa.index[0]]
(aaa.ix[aaa.index[1]] - aaa.ix[aaa.index[0]]) * 0.5
aaa.ix[aaa.index[0]] + (aaa.ix[aaa.index[1]] - aaa.ix[aaa.index[0]]) * 0.5
  

In [110]:
option_table

,Capacity in kW,Raw Cost,Installation Cost as Percent of Raw Cost,Maintenance Cost per Year as Percent of Raw Cost,Lifetime in Years
0,500,52500,25,1,5
1,1000,100000,25,1,5


In [121]:
from infrastructure_planning.exceptions import ExpectedPositive
from invisibleroads_macros.math import divide_safely
from invisibleroads_macros.table import normalize_column_name
from math import ceil
from pandas import read_csv


desired_system_capacity = 750
option_table_path = 'example-diesel-generator-by-capacity.csv'
capacity_column = 'capacity_in_kw'


option_table = read_csv(option_table_path)
option_table.columns = [normalize_column_name(x, '_') for x in option_table.columns]


def prepare_system_cost(desired_system_capacity, option_table, capacity_column):
    x = interpolate_system_cost_parameters(desired_system_capacity, option_table, capacity_column)
    raw_cost = desired_system_capacity * x['raw_cost_per_unit_capacity']
    installation_cost = raw_cost * x['installation_cost_as_percent_of_raw_cost'] / float(100)
    maintenance_cost_per_year = raw_cost * x['maintenance_cost_per_year_as_percent_of_raw_cost'] / float(100)
    replacement_cost_per_year = divide_safely(
        installation_cost, x['lifetime_in_years'],
        ExpectedPositive('lifetime_in_years'))
    return {
        'actual_system_' + capacity_column: int(ceil(desired_system_capacity)),
        'raw_cost': raw_cost,
        'installation_cost': installation_cost,
        'maintenance_cost_per_year': maintenance_cost_per_year,
        'replacement_cost_per_year': replacement_cost_per_year,
    }

prepare_system_cost(desired_system_capacity, option_table, capacity_column)

{'actual_system_capacity_in_kw': 750,
 'installation_cost': 19218.75,
 'maintenance_cost_per_year': 768.75,
 'raw_cost': 76875.0,
 'replacement_cost_per_year': 3843.75}

In [ ]:
"""
    + Name initial variables
    + Draft table
    + Specify final variables
    + Fix oversizing system capacity
        + Draft system capacity algorithm
            + Get generator capacity
            + Get generator raw cost
            + Get generator installation cost as percent of raw cost
            + Get maintenance cost per year as percent of raw cost
            + Get desired system capacity
            _ Add system capacity safety factor
            + Compute raw cost per kw
            + Pick nearest generator capacity and use those costs per kw
            + Round actual capacity to integer
    = Make function
"""

In [ ]:
def prepare_system_cost(
        desired_system_capacity, option_table, capacity_column):
    t = option_table
    # Select option
    eligible_t = t[t[capacity_column] < desired_system_capacity]
    if len(eligible_t):
        t = eligible_t
        # Choose the largest capacity from eligible types
        selected_t = t[t[capacity_column] == t[capacity_column].max()]
    else:
        # Choose the smallest capacity from all types
        selected_t = t[t[capacity_column] == t[capacity_column].min()]
    selected = selected_t.ix[selected_t.index[0]]
    # Get capacity and count
    selected_capacity = selected[capacity_column]
    selected_count = int(ceil(divide_safely(
        desired_system_capacity, selected_capacity,
        ExpectedPositive('selected_capacity'))))
    actual_system_capacity = selected_capacity * selected_count
    # Get costs
    installation_lm_cost = selected_count * selected['installation_lm_cost']
    maintenance_lm_cost_per_year = selected_count * selected[
        'maintenance_lm_cost_per_year']
    replacement_lm_cost_per_year = divide_safely(
        installation_lm_cost, selected['lifetime_in_years'],
        ExpectedPositive('lifetime_in_years'))
    return {
        'desired_system_' + capacity_column: desired_system_capacity,
        'selected_' + capacity_column: selected_capacity,
        'selected_count': selected_count,
        'actual_system_' + capacity_column: actual_system_capacity,
        'installation_lm_cost': installation_lm_cost,
        'maintenance_lm_cost_per_year': maintenance_lm_cost_per_year,
        'replacement_lm_cost_per_year': replacement_lm_cost_per_year,
    }

In [122]:
def interpolate_values(source_table, target_column, target_value):
    t = source_table
    # Get indices of two rows nearest to target value
    difference = t[target_column] - target_value
    sorted_indices = difference.abs().argsort()
    index0 = sorted_indices[0]
    index1 = sorted_indices[1] if len(sorted_indices) > 1 else index0
    # Compute fraction of difference in target column
    fraction = divide_safely(
        t[target_column].ix[index0],
        t[target_column].ix[index1],
        ExpectedPositive(target_column))
    # Interpolate
    return t.ix[index0] + (t.ix[index1] - t.ix[index0]) * fraction


In [123]:
interpolate_values(option_table, capacity_column, 750)

capacity_in_kw                                        750.0
raw_cost                                            76250.0
installation_cost_as_percent_of_raw_cost               25.0
maintenance_cost_per_year_as_percent_of_raw_cost        1.0
lifetime_in_years                                       5.0
dtype: float64